In [1]:
import ee
ee.Initialize()
from s1flood import calc_zscore, mapFloods, floodPalette
import geemap

In [2]:
geometry = ee.Geometry.Polygon(
        [[[33.44259958627015, -18.49604760905407],
          [33.44259958627015, -20.664424742367366],
          [36.43088083627015, -20.664424742367366],
          [36.43088083627015, -18.49604760905407]]]
)

In [3]:
basestart = '2018-01-01'
baseend = '20219-08-01'
targdate = '2019-03-24'

In [4]:
zvv_thd = -2.5
zvh_thd = -2.5

In [5]:
filters = [
  ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
  ee.Filter.listContains("transmitterReceiverPolarisation", "VH"),
  ee.Filter.equals("instrumentMode", "IW"),
  ee.Filter.geometry(geometry),
  ee.Filter.date('2017-01-01', ee.Date(targdate).advance(1, 'day'))
  ]

In [6]:
s1 = ee.ImageCollection("COPERNICUS/S1_GRD").filter(filters)

In [7]:
z_iwasc = calc_zscore(s1, basestart, baseend, 'IW', 'ASCENDING')
z_iwdsc = calc_zscore(s1, basestart, baseend, 'IW', 'DESCENDING')
z = ee.ImageCollection(z_iwasc.merge(z_iwdsc)).sort('system:time_start')

In [8]:
floods = mapFloods(z.mosaic(), zvv_thd, zvh_thd, use_dswe = True)
floods = floods.updateMask(floods.gt(0))

In [9]:
Map = geemap.Map()
Map.setCenter(34.87, -19.82, 10)
Map.addLayer(floods, {'min': 0, 'max': 20, 'palette': floodPalette}, 'Flood Map')

In [10]:
Map

Map(center=[-19.82, 34.87], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…